# tvmc

In [1]:
import logging
from mxnet.gluon.model_zoo.vision import get_model

from tvm import relay
from tvm.driver import tvmc
from tvm.driver.tvmc import TVMCModel, TVMCPackage, TVMCResult

In [2]:
pretrained = True
shape_dict = {'data': (1, 3, 224, 224)}
model_name = 'mobilenet1.0'
out_dir = 'outputs'
logging.basicConfig(filename=f'{out_dir}/{model_name}.log')
logger = logging.getLogger(name='logger')
logger.setLevel(logging.DEBUG)

model = get_model(model_name, pretrained=pretrained)
mod, params = relay.frontend.from_mxnet(model, shape_dict)
model = TVMCModel(mod, params)
tvmc.compile(model, target="llvm", package_path="whatever")
new_package = TVMCPackage(package_path="whatever")
result = tvmc.run(new_package, device='cpu') #Step 3: Run
logger.info(model.mod['main'].astext()) # 记录 mod
print(result.format_times())
result.save(f'{out_dir}/{model_name}_resluts')
model.save(f"{out_dir}/{model_name}.params")

/media/pc/data/4tb/lxw/libs/anaconda3/envs/py38/lib/python3.8/site-packages/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(
2022-11-11 15:08:14.973 INFO load_module /tmp/tmpljccm42j/mod.so


Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   8.4161       8.4096       8.6203       8.2909       0.1058   
               


In [3]:
tuning_records = tvmc.tune(model, target="llvm")

[Task  3/21]  Current/Best:   31.68/ 237.22 GFLOPS | Progress: (476/476) | 206.11 s Done.
[Task  5/21]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/476) | 0.00 s70 s Done.
[Task  5/21]  Current/Best:   11.05/ 286.81 GFLOPS | Progress: (476/476) | 171.35 s Done.
[Task  7/21]  Current/Best:   21.30/ 213.57 GFLOPS | Progress: (476/476) | 245.42 s Done.
[Task  9/21]  Current/Best:   45.21/ 166.23 GFLOPS | Progress: (48/476) | 21.66 s s Done.
[Task 10/21]  Current/Best:   10.10/  89.47 GFLOPS | Progress: (48/476) | 22.26 s s Done.
[Task 11/21]  Current/Best:   80.05/ 222.75 GFLOPS | Progress: (48/476) | 22.69 s s Done.
 Done.
 Done.
[Task 11/21]  Current/Best:   27.32/ 275.92 GFLOPS | Progress: (476/476) | 183.68 s Done.
[Task 13/21]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/476) | 0.00 s40 s Done.
[Task 13/21]  Current/Best:   75.44/ 251.65 GFLOPS | Progress: (476/476) | 169.93 s Done.
[Task 15/21]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/476) | 0.00 s08 s Do

In [4]:
tvmc_package = tvmc.compile(model, target="llvm", tuning_records=tuning_records)
result = tvmc.run(tvmc_package, device="cpu")
print(result)

 Done.
 Done.
 Done.


2022-11-11 17:01:06.201 INFO load_module /tmp/tmpye2ahrm1/mod.so


Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   9.2291       8.9125      11.2151       7.5655       1.1698   
               
Output Names:
 ['output_0']
